In [9]:
import pandas as pd
from mrich import print
from pathlib import Path
from datetime import datetime

In [27]:
data = []
for file in Path("data/InVivo2/spect").glob("*/*.csv"):
    
    df = pd.read_csv(file, skiprows=1)
    df = df[df["Name"] == "Heart"]
    
    date, rat, timepoint, _ = file.name.split("_")
    
    rat = int(rat.removeprefix("R"))
    date = datetime.strptime(date, "%Y%m%d").date()
    timepoint = timepoint.replace("wk", " wk")
    
    d = {
        "Date":date,
        "Timepoint":timepoint,
        "Rat":rat,
    }
    
    for i,row in df.iterrows():
        scan = "30m" if "30m" in row["Series"] else "4h"
        
        key = f"{scan}: {row['Image name']}"
        
        for col in df.columns[3:]:
            d[f"{key} [{col}]"] = row[col]
    
    data.append(d)
    
df = pd.DataFrame(data)

indices = ["Timepoint", "Rat"]
timepoints = ["Baseline", "4 wk", "8 wk", "10 wk", "14 wk", "18 wk", "22 wk", "25 wk"]

df["Timepoint"] = pd.Categorical(df["Timepoint"], categories=timepoints, ordered=True)
    
df = df.sort_values(by=indices)
df = df.set_index(indices)

display(df)

Date                             30m: CT( HU ) [Series]  \
Timepoint Rat                                                                  
Baseline  64   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          69   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          70   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          71   2025-05-03  BR_CTRecon,30m CT Scan (2),FBP,V250,Sl250,F:Co...   
          72   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          73   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          74   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          75   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          77   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          79   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
8 wk      64   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          69   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          70   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          71   2025-05-03  BR_CTRecon,30m CT Scan (2),FBP,V250,Sl250,F:Co...   
          72   2025-05-03  NR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          73   2025-05-03  BR_CTRecon,30m CT Scan (2),FBP,V250,Sl250,F:Co...   
          74   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          75   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          77   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          79   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
25 wk     69   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          70   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          71   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
          72   2025-05-03  BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   

               30m: CT( HU ) [Deviation]  30m: CT( HU ) [Mean]  \
Timepoint Rat                                                    
Baseline  64                       96.17                202.26   
          69                       96.56                202.15   
          70                      100.22                212.47   
          71                      110.65                188.08   
          72                      121.87                180.89   
          73                      105.18                165.23   
          74                      107.07                176.52   
          75                      123.97                176.67   
          77                      115.82                170.09   
          79                      129.47                146.15   
8 wk      64                       96.64                141.21   
          69                       96.87                122.72   
          70                      100.55                162.41   
          71                       98.54                115.49   
          72                      112.77                117.61   
          73                       93.53                112.62   
          74                      244.09                 44.38   
          75                       94.61                130.31   
          77                       90.56                148.04   
          79                       91.12                130.52   
25 wk     69                      185.36                108.51   
          70                      183.97                141.30   
          71                      191.01                117.06   
          72                      101.64                106.56   

               30m: CT( HU ) [Max]  30m: CT( HU ) [Min]  30m: CT( HU ) [Sum]  \
Timepoint Rat                                                                  
Baseline  64                 550.0               -394.0        

In [28]:
df.to_excel("InVivo2_spect_partial.xlsx")